Model Training
- 1.1 Import Data and Required Packages

In [3]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

Import the CSV Data as Pandas DataFrame

In [4]:
df = pd.read_csv(r'C:\Users\mrshe\Downloads\happiness-prediction-using-social-media-main\happiness-prediction-using-social-media-main\Nootbook\data\Mental_Health_and_Social_Media_Balance_Dataset.csv')

In [5]:
df.head()

,User_ID,Age,Gender,Daily_Screen_Time(hrs),Sleep_Quality(1-10),Stress_Level(1-10),Days_Without_Social_Media,Exercise_Frequency(week),Social_Media_Platform,Happiness_Index(1-10)
0,U001,44,Male,3.1,7.0,6.0,2.0,5.0,Facebook,10.0
1,U002,30,Other,5.1,7.0,8.0,5.0,3.0,LinkedIn,10.0
2,U003,23,Other,7.4,6.0,7.0,1.0,3.0,YouTube,6.0
3,U004,36,Female,5.7,7.0,8.0,1.0,1.0,TikTok,8.0
4,U005,34,Female,7.0,4.0,7.0,5.0,1.0,X (Twitter),8.0


Preparing X and Y variables

In [6]:
X = df.drop(columns=['Happiness_Index(1-10)'],axis=1)

In [7]:
X.head()

,User_ID,Age,Gender,Daily_Screen_Time(hrs),Sleep_Quality(1-10),Stress_Level(1-10),Days_Without_Social_Media,Exercise_Frequency(week),Social_Media_Platform
0,U001,44,Male,3.1,7.0,6.0,2.0,5.0,Facebook
1,U002,30,Other,5.1,7.0,8.0,5.0,3.0,LinkedIn
2,U003,23,Other,7.4,6.0,7.0,1.0,3.0,YouTube
3,U004,36,Female,5.7,7.0,8.0,1.0,1.0,TikTok
4,U005,34,Female,7.0,4.0,7.0,5.0,1.0,X (Twitter)


In [9]:
print("Categories in 'gender' variable:     ",end=" " )
print(df['Gender'].unique())

print("Categories in 'Social_Media_Platform' variable:  ",end=" ")
print(df['Social_Media_Platform'].unique())

Categories in 'gender' variable:      ['Male' 'Other' 'Female']
Categories in 'Social_Media_Platform' variable:   ['Facebook' 'LinkedIn' 'YouTube' 'TikTok' 'X (Twitter)' 'Instagram']


In [10]:
y = df['Happiness_Index(1-10)']

In [11]:
y

0      10.0
1      10.0
2       6.0
3       8.0
4       8.0
       ... 
495    10.0
496     9.0
497     8.0
498     9.0
499     7.0
Name: Happiness_Index(1-10), Length: 500, dtype: float64

In [12]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [13]:
X = preprocessor.fit_transform(X)

In [14]:
X.shape

(500, 515)

In [15]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((400, 515), (100, 515))

Create an Evaluate Function to give all metrics after model Training

In [16]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [18]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

   # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.0001
- Mean Absolute Error: 0.0000
- R2 Score: 1.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.9659
- Mean Absolute Error: 0.7961
- R2 Score: 0.6073


Lasso
Model performance for Training set
- Root Mean Squared Error: 1.4313
- Mean Absolute Error: 1.1965
- R2 Score: 0.1110
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 1.4551
- Mean Absolute Error: 1.2525
- R2 Score: 0.1087


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.4431
- Mean Absolute Error: 0.3557
- R2 Score: 0.9148
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.9652
- Mean Absolute Error: 0.7958
- R2 Score: 0.6079


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 0.8570
- Mean Absolute Error: 0.6520
- R2 Score: 0.6813
-----------------------

Results

In [19]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
5,Random Forest Regressor,0.643192
7,CatBoosting Regressor,0.628243
2,Ridge,0.607857
0,Linear Regression,0.607308
3,K-Neighbors Regressor,0.604142
6,XGBRegressor,0.599350
8,AdaBoost Regressor,0.573389
4,Decision Tree,0.486446
1,Lasso,0.108740
